**Some examples on how to use the OMS API**  

The following topics are covered:  
- creating an OMS API instance
- get all available information per run or for a given range of runs
- get all available information per lumisection
- select specific data fields to retrieve
- get L1 and HLT trigger rates

In [ ]:
### imports

# external modules
import matplotlib.pyplot as plt
import importlib

# local modules
import get_oms_data
importlib.reload(get_oms_data)
from get_oms_data import get_oms_api, get_oms_data, get_oms_response_attribute

In [ ]:
# get the omsapi instance

omsapi = get_oms_api()

In [ ]:
# example: get run information for a single run

runnb = 297050
run_info = get_oms_data( omsapi, 'runs', runnb )
print(run_info)

In [ ]:
# example: get only specific info for a single run

runnb = 297050
run_info_spec = get_oms_data( omsapi, 'runs', runnb, attributes=['l1_rate','l1_menu'] )
print(run_info_spec)
print(run_info_spec['data'])
print(run_info_spec['data'][0]['attributes'])

In [ ]:
# example: get specific run information for multiple runs

runnb = (297050,297056)
run_info_mult = get_oms_data( omsapi, 'runs', runnb, attributes=['l1_rate','l1_menu'] )
print(run_info_mult)
l1_rate = get_oms_response_attribute( run_info_mult, 'l1_rate' )
print(l1_rate)

In [ ]:
### example: get available information per lumisection

runnb = 297050
lsfilter = {'attribute_name':'lumisection_number','value':1,'operator':'EQ'}
ls_info = get_oms_data( omsapi, 'lumisections', runnb, extrafilters=[lsfilter] )
attrs = ls_info['data'][0]['attributes']
for key,val in attrs.items():
    print('{} -> {}'.format(key,val))

In [ ]:
# example: get pileup and luminosity information per lumisection

runnb = 297050
ls_info = get_oms_data( omsapi, 'lumisections', runnb, attributes=['lumisection_number','pileup','delivered_lumi','recorded_lumi'] )
#print(ls_info)
ls_nbs = get_oms_response_attribute(ls_info,'lumisection_number')
pileup = get_oms_response_attribute(ls_info,'pileup')
del_lumi = get_oms_response_attribute(ls_info,'delivered_lumi')
rec_lumi = get_oms_response_attribute(ls_info,'recorded_lumi')

plt.figure()
plt.plot(ls_nbs,pileup)
plt.figure()
plt.plot(ls_nbs,del_lumi,color='b')
plt.plot(ls_nbs,rec_lumi,color='g')

In [ ]:
### example: get L1 trigger rate per lumisection

import numpy as np

runnb = 297050
#lsfilter = {'attribute_name':'lumisection_number','value':10,'operator':'LT'}
ls_info = get_oms_data( omsapi, 'l1triggerrates', runnb, extraargs={'group[granularity]':'lumisection'} )
#print(ls_info)
attrs = ls_info['data'][0]['attributes']
for attr in attrs: print('{} -> {}'.format(attr,attrs[attr]))
lsnbs = get_oms_response_attribute(ls_info,'first_lumisection_number')
l1_rate = get_oms_response_attribute(ls_info,'l1a_physics')
l1_rate = [el['rate'] for el in l1_rate]
#print(l1_rate)
print(np.mean(l1_rate))
plt.figure()
plt.plot(lsnbs,l1_rate)

In [ ]:
### example: get HLT trigger rate per lumisection

runnb = 297050
ls_info = get_oms_data( omsapi, 'hltpathinfo', runnb, attributes=['run_number','path_name'] )
#print(ls_info)
print('available path names:')
print(get_oms_response_attribute(ls_info,'path_name'))

path_filter = {'attribute_name':'path_name','value':'HLT_ZeroBias_v5','operator':'EQ'}
ls_info = get_oms_data( omsapi, 'hltpathrates', runnb, extraargs={'group[granularity]':'lumisection'}, extrafilters=[path_filter] )
#print(ls_info)
lsnbs = get_oms_response_attribute(ls_info,'first_lumisection_number')
hlt_rate = get_oms_response_attribute(ls_info,'rate')
#print(hlt_rate)
plt.figure()
plt.plot(lsnbs,hlt_rate)

In [ ]:
### example: check if 'cms_active' attribute is same as DCS-json

import os
import sys
sys.path.append(os.path.abspath('../utils/'))
import json_utils as jsonu
import plot_utils as pu

runnb = 306458
ls_info = get_oms_data( omsapi, 'lumisections', runnb, attributes=['lumisection_number','cms_active'] )
lsnbs = get_oms_response_attribute(ls_info,'lumisection_number')
cms_active = get_oms_response_attribute(ls_info,'cms_active')
print(cms_active)
runnbs = [runnb]*len(lsnbs)
dcson = jsonu.isdcson( runnbs, lsnbs )
print(dcson)
pu.plot_hists([cms_active],colorlist=['blue'], labellist=['cms_active'],
              xaxtitle='lumisection number', yaxtitle=None, bkgcolor=dcson, bkgcmap='cool')